# Table of Contents
 <p>

In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import glob
import os
from geopy.distance import vincenty

In [2]:
#data_path = "/Users/jennavergeynst/Documents/Altusried/VPS_Test_Results_VEMCO/data/"
# for binder:
data_path = '../data/'


In [3]:
walls = pd.read_csv(data_path+'QGIS/studysite.csv')
test_pos = pd.read_csv(data_path+'fish-csv/all.csv', parse_dates=['Time'], infer_datetime_format=True)
gps_data = pd.read_csv(data_path+'Alle_GPS_tracks_10aug2017.csv', parse_dates=['time'], infer_datetime_format=True).sort_values(by='time').set_index('time')
Overview_coo_timing_range_test = pd.read_excel(data_path+'Overview_coördinates_and_timing_range_test_set_ups.xlsx', sheet_name='Thursday', skiprows=6)

In [ ]:
def find_gps_coo(row):
    return gps_data.iloc[gps_data.index.get_loc(test_pos.Time[row], method='nearest')]

In [ ]:
test_pos['LAT_GPS'] = [find_gps_coo(i).latitude for i in np.arange(len(test_pos))]
test_pos['LON_GPS'] = [find_gps_coo(i).longitude for i in np.arange(len(test_pos))]

test_pos['error'] = test_pos.apply(lambda row: vincenty((row.Latitude, row.Longitude), (row.LAT_GPS, row.LON_GPS)).m, axis=1)

In [ ]:
Overview_coo_timing_range_test[Overview_coo_timing_range_test.Tag_position_track_code.isin(['TR'+str(n) for n in range(4,11)])]

In [ ]:
# onderstaande geeft enkel tracks als ik 2 uur aftrek van de uren gegeven in Overview

sub = test_pos.set_index('Time')['2017-08-10 14:18':'2017-08-10 14:22'] 

fig,ax = plt.subplots(figsize=(6,6))
walls.plot('x', 'y', c='black', ax=ax, label='')
sub.plot('Longitude', 'Latitude', ax=ax, label='VPS', marker='.')
sub.plot('LON_GPS', 'LAT_GPS', ax=ax, label='GPS', marker='.')
plt.legend()

ax.set_title('Mean error on track: {:.2f} m'.format(sub.error.mean()))